### 필요한 파일
- tag_list.txt (Train에 있는 태그리스트)
- genre_tag1.json (세부장르별 가장 많이 언급된 태그 1개씩 붙어있는 데이터프레임)
- testval_tag.txt (test와 validation에만 있는 태그리스트)
- tagsong.json (현민님이 보내주신 파일에서 tag별 가장 많이 언급된 노래 10개까지만 뽑았습니다)

In [1]:
val = pd.read_json('../raw_data/val.json', encoding="Utf-8", typ='frame')
test = pd.read_json('../raw_data/test.json', encoding="Utf-8", typ='frame')
song_meta = pd.read_json('../raw_data/song_meta.json', encoding="Utf-8", typ='frame')
train = pd.read_json('../raw_data/train.json', encoding="Utf-8", typ='frame')

### import

In [2]:
import pickle
from collections import Counter
from tqdm.notebook import tqdm
from konlpy.tag import Okt;
import re
import time
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.linalg import norm

In [3]:
# validation, test에만 있는 태그
with open("testval_tag.txt","rb") as fr:
    testval_tag = pickle.load(fr)

### Validation, Test 데이터프레임에서 train에 없는 태그 모두 지워줌

In [4]:
for tagls in val['tags']:
    for tag in reversed(tagls):
        if tag in testval_tag:
            tagls.remove(tag)

In [5]:
for tagls in test['tags']:
    for tag in reversed(tagls):
        if tag in testval_tag:
            tagls.remove(tag)

In [6]:
val[val['tags'].apply(lambda x: len(x) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23003,[],127349,,"[474446, 111476, 241237, 70500, 577313, 145058...",14,2018-02-02 17:43:37.000
23006,[],92067,,"[486938, 154124, 70969, 596414, 243733, 557848...",9,2013-02-16 01:07:41.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


### 문장간의 거리구하는 함수

In [7]:
def get_dist(v1, v2):
    delta = v1- v2
    return sp.linalg.norm(delta.toarray())

### 태그가 없는 플레이리스트의 태그채우기 (30-35분 소요)
- 1. 태그가 없는 플레이리스트중에 노래가 있는 플레이리스트는 그 노래들의 세부장르를 모아 가장 많이 언급된 세부장르를 뽑아 그 세부장르와 같이 가장 많이 언급된 태그 하나를 추가
- 2. 태그도 없고 노래도 없는 플레이리스트의 제목에서 명사들을 뽑고 그 명사가 Train의 태그리스트에 존재하면 추가
- 3. 태그가 안채워진 플레이리스트가 있다면 이번엔 형태소로 뽑아 그 형태소가 Train의 태그리스트에 존재하면 추가
- 4. 그래도 안채워진 플레이리스트가 있다면 Train의 태그리스트의 태그들을 모두 TFIDF를 적용하여 벡터화 시킨 후 플레이리스트의 제목과 거리를 비교하여 가장 가까운 태그를 추가
- 5. 그래도 안채워진 플레이리스트가 있다면(제목도 없는 플레이리스트) 가장 많이 언급된 태그(기분전환)를 추가

In [8]:
def get_tags(df):
    genre_tag = pd.read_json('genre_tag1.json')
    no_tag = df[df['tags'].apply(lambda x: len(x) == 0)]
    
    # 노래들이 가지고 있는 장르들을 한 리스트에 추가
    gnr_all = []
    for song_ls in no_tag['songs']:
        gnr_ls = []
        for song in song_ls:
            for i in song_meta[song_meta['id']==song]['song_gn_dtl_gnr_basket'][song]:
                gnr_ls.append(i)
        gnr_all.append(gnr_ls)
    
    no_tag['genre'] = gnr_all
    no_tag['genre_cnt'] = no_tag['genre'].apply(lambda x: Counter(x).most_common(1))
    no_tag.drop(['genre'], axis=1, inplace=True)
    no_tag['genre'] = no_tag['genre_cnt'].apply(lambda x: x[0][0] if len(x) >= 1 else x)
    
    # 장르컬럼에서 태그를 가져와서 추가
    for gnr, i in zip(no_tag['genre'], no_tag.index):
        if len(gnr) > 1:
            no_tag['tags'][i].append(genre_tag[genre_tag['song_gn_dtl_gnr_basket']==gnr]['tag'].item())
    
    print("장르로 태그 추가하기 완료")
    
    # 노래 제목만 있는 데이터들의 태그 채우기
    no_song = no_tag[no_tag['tags'].apply(lambda x: len(x) == 0)]
    
    # 명사, 형태소 추가
    
    t = Okt()
    no_song['noun'] = no_song['plylst_title'].apply(t.nouns)
    no_song['morph'] = no_song['plylst_title'].apply(t.morphs)
    
    # Train 데이터에 있는 전체 태그리스트를 불러옴
    with open("tag_list.txt","rb") as fr:
        all_tag = pickle.load(fr)
    
    # 명사를 추출해서 전체 태그리스트에 존재한다면 추가
    for noun_ls, i in zip(no_song['noun'], no_song['noun'].index):
        for noun in noun_ls:
            if noun in all_tag:
                if noun not in no_song['tags'][i]:
                    no_song['tags'][i].append(noun)
    
    # 명사로 안채워진 tag가 있다면 형태소로 다시 확인해서 추가
    no_song1 = no_song[no_song['tags'].apply(lambda x: len(x)==0)]
    for morph_ls, i in zip(no_song1['morph'], no_song1['morph'].index):
        for morph in morph_ls:
            if morph in all_tag:
                if morph not in no_song1['tags'][i]:
                    no_song1['tags'][i].append(morph)
    
    print("명사, 형태소로 태그 채우기 완료")
    # 그래도 안채워진 데이터 채우기
    last = no_song1[no_song1['tags'].apply(lambda x: len(x) == 0)]
    
    test_plylist = []
    for i in last['plylst_title']:
        test_plylist.append(i)
    
    tokens = [t.morphs(row) for row in all_tag]
    # 형태소 분석이 된 리스트를 다시 하나로 합치기
    tokens_vectorize = []

    for token in tokens:
        sentence = ''
        for word in token:
            sentence = sentence + ' ' + word
        tokens_vectorize.append(sentence[1:])
    
    # 특수문자 제거 및 띄어쓰기는 한개로
    tokens_vectorize1 = [re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9 ]', '', i) for i in tokens_vectorize]
    tokens_vectorize = [re.sub(' +', ' ', i) for i in tokens_vectorize1]
    
    # TFIDF 적용 (하나는 단어단위, 하나는 형태소 단위)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer1 = TfidfVectorizer(analyzer='char')
    
    X = tfidf_vectorizer.fit_transform(tokens_vectorize)
    X1 = tfidf_vectorizer1.fit_transform(tokens_vectorize)
    
    test_plylist = [t.morphs(row) for row in test_plylist]
    
    test_vectorize = []

    for plylist in test_plylist:
        sentence = ''
        for word in plylist:
            sentence = sentence + ' ' + word
        test_vectorize.append(sentence[1:])
        
    test_vectorize = [re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9 ]', '', i) for i in test_vectorize]
    test_vectorize = [re.sub(' +', ' ', i) for i in test_vectorize]
    
    # 적용
    new = tfidf_vectorizer.transform(test_vectorize)
    
    distls = []
    for i in range(0, len(test_vectorize)):
        dist = [get_dist(each, new[i]) for each in X]
        distls.append(dist)

        
    # 플레이리스트와 tag들간의 거리가 가장 짧은 태그 하나 추가
    for i, j in zip(last['tags'].index, range(0, len(test_vectorize))):
        if all_tag[distls[j].index(min(distls[j]))] not in df['tags'][i]:
            last['tags'][i].append(all_tag[distls[j].index(min(distls[j]))])
    
    # 리스트안에 아무것도 없는 string은 제거
    last['tags'].apply(lambda x: x.remove('') if '' in x else x)
    
    print("TFIDF 단어 단위로 태그 채우기 완료")
    # 아직도 안채워진 태그
    last1 = last[last['tags'].apply(lambda x: len(x) == 0)]
    
    test_plylist1 = []
    for i in last1['plylst_title']:
        test_plylist1.append(i)
        
    test_plylist1 = [t.morphs(row) for row in test_plylist1]
    
    test_vectorize1 = []

    for plylist in test_plylist1:
        sentence = ''
        for word in plylist:
            sentence = sentence + ' ' + word
        test_vectorize1.append(sentence[1:])
        
    test_vectorize1 = [re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9 ]', '', i) for i in test_vectorize1]
    test_vectorize1 = [re.sub(' +', ' ', i) for i in test_vectorize1]
    
    new1 = tfidf_vectorizer1.transform(test_vectorize1)
    
    distls1 = []
    for i in range(0, len(test_vectorize1)):
        dist1 = [get_dist(each, new1[i]) for each in X1]
        distls1.append(dist1)
    
    for i, j in zip(last1['tags'].index, range(0, len(test_vectorize1))):
        if all_tag[distls[j].index(min(distls[j]))] not in df['tags'][i]:
            last1['tags'][i].append(all_tag[distls1[j].index(min(distls1[j]))])
    
    last1['tags'].apply(lambda x: x.remove('') if '' in x else x)
    
    print("TFIDF Char 단위로 태그 채우기 완료")
    
    # 그래도 안채워지는 것들은 그냥 제일 언급이 많이 된 태그를 추가
    last2 = last1[last1['tags'].apply(lambda x: len(x) == 0)]
    
    last2['tags'].apply(lambda x: x.append(Counter(list(itertools.chain.from_iterable(list(train['tags'])))).most_common(1)[0][0]))
    
    return df

In [9]:
validation = get_tags(val)
validation

장르로 태그 추가하기 완료
명사, 형태소로 태그 채우기 완료
TFIDF 단어 단위로 태그 채우기 완료
TFIDF Char 단위로 태그 채우기 완료


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[OST],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[턴테이블],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[발라드],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[발라드],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[CCM],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[팝],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[클래식],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [10]:
testdf = get_tags(test)
testdf

장르로 태그 추가하기 완료
명사, 형태소로 태그 채우기 완료
TFIDF 단어 단위로 태그 채우기 완료
TFIDF Char 단위로 태그 채우기 완료


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[댄스],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[일렉],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000


In [11]:
validation[validation['tags'].apply(lambda x: len(x) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [12]:
testdf[testdf['tags'].apply(lambda x: len(x) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date


### 노래 채우기 (5분 소요)
- 플레이리스트 제목으로 노래를 뽑는 것은 정확도가 떨어져서 그냥 현민님이 처음에 하셨던 카운트로 했습니다. 일단 tagsong 파일에서 상위 10개의 노래만 뽑아서 추가했습니다. (10개가 없는 것들은 10개가 안될수도 있어요)

In [13]:
def get_song(df):
    nosong = df[df['songs'].apply(lambda x: len(x) == 0)]
    tagsong = pd.read_json('tagsong.json')
    
    # songtag의 태그와 매칭시켜 노래 채우기
    for tagls, i in zip(nosong['tags'], nosong['tags'].index):
        for tag in tagls:
            if tag in tagsong['tags'].unique():
                for song in tagsong[tagsong['tags'] == tag]['songs_num'].item():
                    if song not in nosong['songs'][i]:
                        nosong['songs'][i].append(song)
                   
    return df

In [14]:
validation1 = get_song(validation)
validation1

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[OST],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[턴테이블],131447,앨리스테이블,"[572238, 233146, 61064, 336013, 611334, 359175...",1,2014-07-16 15:24:24.000
2,[발라드],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[발라드],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[CCM],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[팝],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[클래식],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [15]:
testdf1 = get_song(testdf)
testdf1

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[댄스],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[일렉],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000


In [16]:
validation1[validation1['songs'].apply(lambda x: len(x) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [17]:
testdf1[testdf1['songs'].apply(lambda x: len(x) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date


### 저장

In [18]:
validation1.to_json('validation.json')

In [19]:
testdf1.to_json('testdf.json')

In [20]:
val_data = pd.read_json('validation.json')